In [0]:
# Install LGBM
!pip install lightgbm

In [16]:
!pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:211: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)


In [13]:
!pip install -qq lightgbm --install-option=--gpu

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:211: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)


In [0]:
# Or GPU version
!apt-get -qq install --no-install-recommends nvidia-375
!apt-get -qq install --no-install-recommends nvidia-opencl-icd-375 nvidia-opencl-dev opencl-headers
!apt-get -qq install --no-install-recommends git cmake build-essential libboost-dev libboost-system-dev libboost-filesystem-dev    
!pip install -qq lightgbm --install-option=--gpu

In [17]:
# Running on Colab
from google.colab import drive
drive.mount('/drive/')
#!df -h

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive/
Filesystem      Size  Used Avail Use% Mounted on
overlay          40G   13G   25G  34% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G  4.0K  6.4G   1% /var/colab
/dev/sda1        46G   14G   32G  30% /etc/hosts
shm             6.0G     0  6.0G   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            40G   14G   26G  36% /drive


In [0]:
!cp submission_LGBM.csv "/drive/My Drive/Study/ADA"

In [0]:
!apt-get install git-lfs

!git lfs clone https://github.com/jokerkeny/LGBM-NYCTaxiTripDuration.git
!ls LGBM-NYCTaxiTripDuration/gdata -l

In [3]:
import os
os.chdir("LGBM-NYCTaxiTripDuration/LGBM")
!ls

FileNotFoundError: ignored

In [1]:
# Grid Search Switch
CV=True

In [2]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.datasets import  make_classification
import numpy as np
import time
import gc

In [20]:
# 加载数据
print('Load data...')

train_raw=pd.read_csv('../gdata/train.csv')

# Reduce the cv dataset
train_raw=train_raw.sample(frac=0.1)

Train_Master=train_raw.drop('Unnamed: 0',axis=1)
X_train=Train_Master.drop(['log_trip_duration'],axis=1)
y_train=Train_Master['log_trip_duration']
del train_raw
del Train_Master

X_train,X_test,y_train,y_test =train_test_split(X_train,y_train,test_size=0.2)

# Example For iris 
# iris = load_iris()
# data=iris.data
# target = iris.target
# X_train,X_test,y_train,y_test =train_test_split(data,target,test_size=0.2)

# df_train = pd.read_csv('../regression/regression.train', header=None, sep='\t')
# df_test = pd.read_csv('../regression/regression.test', header=None, sep='\t')
# y_train = df_train[0].values
# y_test = df_test[0].values
# X_train = df_train.drop(0, axis=1).values
# X_test = df_test.drop(0, axis=1).values

gc.collect()

Load data...


63

In [0]:
# 网格搜索，参数优化 1
if(CV):

    estimator = lgb.LGBMRegressor()

    param_grid1 = {
        'learning_rate': [0.09,0.08],
        'n_estimators': [120,130,140],
        'num_leaves':[130]
    }

    gbm = GridSearchCV(estimator, param_grid1)

    gbm.fit(X_train, y_train)

    print('Best parameters found by grid search are:', gbm.best_params_)

In [26]:
# 网格搜索，参数优化 2
if(CV):

    estimator = lgb.LGBMRegressor()
    
    param_grid2 = {
        'learning_rate': [0.08,0.07],
        'n_estimators': [150,180],
        'num_leaves':[130]
    }

    gbm = GridSearchCV(estimator, param_grid2)

    gbm.fit(X_train, y_train)

    print('Best parameters found by grid search are:', gbm.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.07, 'n_estimators': 180, 'num_leaves': 130}


## Test on training data

In [0]:
gc.collect()
t0=time.time()
print('Start training...')
# 创建模型，训练模型
gbm_bst = lgb.LGBMRegressor(objective='regression',num_leaves=31,learning_rate=0.1,n_estimators=40)
gbm_bst.fit(X_train, y_train,eval_set=[(X_test, y_test)],eval_metric='l1',early_stopping_rounds=5)

print('Start predicting...')
# 测试机预测
y_pred = gbm_bst.predict(X_test, num_iteration=gbm_bst.best_iteration_)
# 模型评估
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

# feature importances
print('Feature importances:', list(gbm_bst.feature_importances_))
t1=time.time()
print('elapse time: {}'.format(t1-t0))

## Final Train & Test for Kaggle Test

In [3]:
gc.collect()
train_raw=pd.read_csv('../gdata/train.csv')

# Reduce dataset
train_raw=train_raw.sample(frac=0.1)

Train_Master=train_raw.drop('Unnamed: 0',axis=1)

X_train=Train_Master.drop(['log_trip_duration'],axis=1)
y_train=Train_Master['log_trip_duration']
del train_raw
del Train_Master

X_train,X_test,y_train,y_test =train_test_split(X_train,y_train,test_size=0.2)



# Final training
t0=time.time()
gbm_final = lgb.LGBMRegressor(objective='regression',num_leaves=130,learning_rate=0.09,n_estimators=130)
gbm_final.fit(X_train, y_train,eval_set=[(X_test, y_test)],eval_metric='l1',early_stopping_rounds=5)

print('Start predicting...')
# 测试机预测
y_pred = gbm_final.predict(X_test, num_iteration=gbm_final.best_iteration_)
# 模型评估
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

# feature importances
print('Feature importances:', list(gbm_final.feature_importances_))
t1=time.time()
print('elapse time: {}'.format(t1-t0))

[1]	valid_0's l1: 0.556864	valid_0's l2: 0.51667
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.524081	valid_0's l2: 0.459015
[3]	valid_0's l1: 0.495095	valid_0's l2: 0.411046
[4]	valid_0's l1: 0.469551	valid_0's l2: 0.371482
[5]	valid_0's l1: 0.447086	valid_0's l2: 0.33834
[6]	valid_0's l1: 0.427299	valid_0's l2: 0.310732
[7]	valid_0's l1: 0.410118	valid_0's l2: 0.287903
[8]	valid_0's l1: 0.395117	valid_0's l2: 0.268819
[9]	valid_0's l1: 0.382132	valid_0's l2: 0.252869
[10]	valid_0's l1: 0.370752	valid_0's l2: 0.239358
[11]	valid_0's l1: 0.361095	valid_0's l2: 0.228155
[12]	valid_0's l1: 0.352688	valid_0's l2: 0.218799
[13]	valid_0's l1: 0.345432	valid_0's l2: 0.210984
[14]	valid_0's l1: 0.339059	valid_0's l2: 0.204229
[15]	valid_0's l1: 0.333595	valid_0's l2: 0.198557
[16]	valid_0's l1: 0.328952	valid_0's l2: 0.193883
[17]	valid_0's l1: 0.324818	valid_0's l2: 0.189772
[18]	valid_0's l1: 0.321435	valid_0's l2: 0.186423
[19]	valid_0's l1: 0.318308	vali

In [4]:
# Save Model
import pickle
pickle.dump(gbm_final,open('LGBM_Model.dat','wb'))

In [5]:
# read test data
test_X=pd.read_csv('../gdata/test.csv')
test_X=test_X.drop('Unnamed: 0',axis=1)
# print(test_X.head())

In [6]:
prediction=gbm_final.predict(test_X)
# recover log transformation log(y+1)
pred = np.exp(prediction) - 1

In [7]:
# add id
original_test=pd.read_csv('../new-york-city-taxi-with-osrm/test.csv')
Test_id=original_test['id']

In [8]:
submission = pd.concat([Test_id, pd.DataFrame(pred)], axis=1)
submission.columns = ['id','trip_duration']
submission['trip_duration'] = submission.apply(lambda x : 1 if (x['trip_duration'] <= 0) else x['trip_duration'], axis = 1)
submission.to_csv("submission_LGBM.csv", index=False)

## Result on Kaggle
### First
Dataset:
training fraction: 0.1 * 0.8

Hyperparameter:
num_leaves=31
learning_rate=0.1
n_estimators=40


Name
Submitted
Wait time
Execution time
Score

submission_LGBM.csv
7 minutes ago
1 seconds
4 seconds
0.45924

### Second
Dataset:
Fraction: 0.5 * 0.8

Score: 0.46887

### Third
Dataset:
Fraction: 0.1 * 0.8
Score: 0.47533